## Semua Dataset

In [11]:
#from google.colab import drive
#drive.mount('/content/drive')

In [12]:
%pip install mediapipe opencv-python

import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp

Note: you may need to restart the kernel to use updated packages.


In [13]:
import os, cv2, numpy as np, pandas as pd
import mediapipe as mp
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np
import pandas as pd

# Inisialisasi MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

# Fungsi ekstrak setiap 5 frame
def extract_pose_every_5frames(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Gagal membuka video: {video_path}")
        return None

    frame_interval = 5
    selected_frames = []
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = pose.process(image)
            if result.pose_landmarks:
                keypoints = []
                for lm in result.pose_landmarks.landmark:
                    keypoints.extend([lm.x, lm.y, lm.z])
                selected_frames.append(keypoints)

        frame_idx += 1

    cap.release()
    return np.mean(selected_frames, axis=0) if selected_frames else None

# Loop dataset
data = []
labels = []

dataset_path = "D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\data"

for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)
    if not os.path.isdir(label_path):
        continue

    print(f"\n📁 Kelas: {label}")
    for filename in os.listdir(label_path):
        if filename.endswith(".mp4"):
            video_path = os.path.join(label_path, filename)
            print(f"🎥 Memproses: {video_path}")

            keypoints = extract_pose_every_5frames(video_path)

            if keypoints is not None:
                data.append(keypoints)
                labels.append(label)
                print(f"✅ Pose berhasil dari: {filename}")
            else:
                print(f"⚠️ Pose tidak ditemukan di: {filename}")

# Simpan ke CSV
df = pd.DataFrame(data)
df['label'] = labels
csv_path = "D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\model2\pose_dataset_5frame.csv"
df.to_csv(csv_path, index=False)
print(f"\n📦 Dataset berhasil disimpan ke: {csv_path}")



📁 Kelas: Ankle Circles
🎥 Memproses: D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\data\Ankle Circles\ankle circles 1.mp4
✅ Pose berhasil dari: ankle circles 1.mp4

📁 Kelas: Arm Circles
🎥 Memproses: D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\data\Arm Circles\arm circles 1.mp4
✅ Pose berhasil dari: arm circles 1.mp4
🎥 Memproses: D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\data\Arm Circles\arm circles 2.mp4
✅ Pose berhasil dari: arm circles 2.mp4

📁 Kelas: Breathing Exercise
🎥 Memproses: D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\data\Breathing Exercise\breathing exercise 1.mp4
✅ Pose berhasil dari: breathing exercise 1.mp4
🎥 Memproses: D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\data\Breathing Exercise\breathing exercise 2.mp4
✅ Pose berhasil dari: breathing exercise 2.mp4
🎥 Memproses: D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\data\Breathing Exercise\breathing exercise 3.mp4
✅ Pose berhasil

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# 1. Baca file CSV, bukan .h5!
csv_path = "pose_dataset_5frame.csv"
df = pd.read_csv(csv_path)

# 2. Pisahkan fitur dan label
X = df.drop("label", axis=1).values
y = df["label"].values

# 3. Encode label
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 4. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 5. Bangun model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

# 6. Compile & train
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

# # 7. Simpan model
# model.save("/content/drive/MyDrive/Semester 6/Capstone/pose_model.h5")

Epoch 1/100


d:\Semester 6\program\Python310\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.9674 - val_accuracy: 0.0000e+00 - val_loss: 2.8336
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.1111 - loss: 2.8429 - val_accuracy: 0.0000e+00 - val_loss: 2.8352
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.0556 - loss: 2.7421 - val_accuracy: 0.0000e+00 - val_loss: 2.8446
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.0000e+00 - loss: 2.9485 - val_accuracy: 0.0000e+00 - val_loss: 2.8527
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.0556 - loss: 2.7843 - val_accuracy: 0.0000e+00 - val_loss: 2.8505
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.1111 - loss: 2.6744 - val_accuracy: 0.0000e+00 - val_loss: 2.8498
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.1667 - loss: 2.5813 - val_accuracy: 0.0000e+00 - val_loss: 2.8545
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.1111 - loss: 2.5291 - val

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np # Pastikan numpy sudah diimpor

# Pastikan data sudah ada dan valid
if len(X) == 0:
    raise ValueError("Data pose tidak ditemukan. Pastikan video mengandung pose yang terdeteksi.")

# Pindahkan validasi shape sebelum reshape
# Memastikan bahwa setiap sampel data pose memiliki 66 fitur (33 x, 33 y)
if X.shape[1] != 66:
    raise ValueError(f"Jumlah fitur ({X.shape[1]}) tidak sesuai dengan yang diharapkan (66) sebelum reshape.")

# Reshape agar sesuai input CNN
# Mengubah shape dari (jumlah_sampel, 66) menjadi (jumlah_sampel, 11, 6, 1)
# Ini dilakukan untuk menyiapkan data sebagai input gambar 11x6 dengan 1 channel
X = X.reshape(-1, 11, 6, 1)

# Split data menjadi train dan test, stratify agar kelas seimbang
# Stratify memastikan bahwa distribusi kelas di set train dan test serupa dengan distribusi kelas di data asli
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Data training: {X_train.shape}, Data testing: {X_test.shape}")

ValueError: Jumlah fitur (99) tidak sesuai dengan yang diharapkan (66) sebelum reshape.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np # Pastikan numpy diimpor jika num_classes dihitung di sini

# Pastikan num_classes dihitung dari data y yang sudah ada
# Jika y belum didefinisikan di sel ini, pastikan sel sebelumnya (pengumpulan data) sudah dijalankan.
if 'y' not in locals():
    raise NameError("Variabel 'y' (label data) belum didefinisikan. Pastikan sel pengumpulan data telah dijalankan.")

num_classes = len(np.unique(y))

# Bangun model CNN dengan padding 'same'
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(11, 6, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary() # Hapus karakter 'a' di sini

# Training
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32
)

# Evaluasi
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 11, 6, 32)      │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 3, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 5, 3, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 960)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       123,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 142,469 (556.52 KB)

 Trainable params: 142,469 (556.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.2498 - loss: 1.5995 - val_accuracy: 0.2767 - val_loss: 1.5761
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2930 - loss: 1.5646 - val_accuracy: 0.3208 - val_loss: 1.5285
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3456 - loss: 1.5142 - val_accuracy: 0.4654 - val_loss: 1.4402
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4037 - loss: 1.4358 - val_accuracy: 0.4969 - val_loss: 1.3215
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4479 - loss: 1.3378 - val_accuracy: 0.5346 - val_loss: 1.2089
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5161 - loss: 1.2311 - val_accuracy: 0.5535 - val_loss: 1.1499
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5607 - loss: 1.1517 - val_accuracy: 0.5786 - val_loss: 1.0730
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5232 - loss: 1.1808 - val_accuracy: 0.5723 - v

In [ ]:
# Simpan model
model.save("D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\model2\model.h5")

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("D:\Semester 6\PEMROGRAMAN SISTEM CERDAS 2\capstone_mp_model\model2\model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmp_fa6fmjj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 11, 6, 1), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  134290348866448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134290348863376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134290348867216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134290348865680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134290348866640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134290348863760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134290345670992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134290345670224: TensorSpec(shape=(), dtype=tf.resource, name=None)
